In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
pd.set_option('display.max_rows',None)
import time
from selenium.webdriver.common.by import By


from flask import jsonify
from flask import request
from flask_api import FlaskAPI
from flask_restful import Api 
from sqlalchemy import create_engine
import json


In [2]:
name = []
role = []
address = []
country = []
telephone = []
mobile = []
fax = []
email = []
final_list=[]
link = []
data = []


In [3]:
def get_scrap_data():
    #working
    driverPath = r"C:\Users\Merit\Desktop\chromedriver.exe"

    driver = webdriver.Chrome(executable_path=driverPath)
    driver.maximize_window()

    driver.get("https://www.freshfields.com/en-gb/contacts/find-a-lawyer/?Name=&t=&Service=97&Role=&Location=&Office=&Industry=")
    driver.implicitly_wait(10)
    time.sleep(2)
    driver.find_element(By.XPATH, '//button[text()="Accept All Cookies"]').click()
    n = driver.find_elements_by_xpath("//div[contains(@class,'row fal-flex')]//following::a[contains(@alt,'pageCount')]")

    try:
        for i in range(1,len(n)+1):
            x = driver.find_elements_by_xpath("//div[contains(@class,'row fal-flex')]/div")
            for j in range(1,len(x)+1):
                l = []
                link.append(driver.find_element_by_xpath("//div[contains(@class,'row fal-flex')]/div["+str(j)+"]/a[1]").get_attribute("href"))
                name = driver.find_element_by_xpath("//div[contains(@class,'row fal-flex')]/div["+str(j)+"]/a/p").text.split("\n")
                driver.find_element_by_xpath("//div[contains(@class,'row fal-flex')]/div["+str(j)+"]/a[1]").click()
                contact = driver.find_elements_by_xpath("//div[contains(@class,'profile-details__related-contact-details')]")
                for k in contact:
                    l.append(k.text.split("\n"))
                if(l[1] == ['']):
                    l = []
                    contact = driver.find_elements_by_xpath("//address")
                    country = driver.find_element_by_xpath("//address/preceding-sibling::h3")
                    mail = driver.find_element_by_xpath("//address//following::a[1]")
                    for lst in contact:
                        l.append(lst.text.split("\n"))
                    l = l[0]
                    l.append(mail.text)
                    l.insert(0,country.text)
                else:
                    l = l[1]
                final_list.append(name + l)
                driver.back()
            driver.find_element_by_link_text("Next").click()
        driver.close()

    except:
        driver.close()
    return final_list




In [4]:
def get_dataframe():
    lst = get_scrap_data()
    for i in lst:
        name.append(i.pop(0))
        role.append(i.pop(0))
        country.append(i.pop(0))

        email.append(i.pop())

        fax.append('+'+i.pop().replace('F +',''))

        x = i.pop()
        if('M +' in x):
            mobile.append('+'+x.replace('M +',''))
        else:
            mobile.append('null')
            telephone.append('+'+x.replace('T +',''))
            address.append(" ".join(i))
            continue

        telephone.append('+'+i.pop().replace('T +',''))
        address.append(" ".join(i))
    
#     data = list(zip(link,name,role,address,country,mobile,telephone,fax,email))
    df = pd.DataFrame(list(zip(link,name,role,address,country,mobile,telephone,fax,email)), columns = ["URL","Name","Role","Address","Country","Mobile","Telephone","Fax","Email"])
    
    return df


In [5]:
data = get_dataframe()


In [6]:
data

,URL,Name,Role,Address,Country,Mobile,Telephone,Fax,Email
0,https://www.freshfields.com/en-gb/contacts/fin...,Fares Al-Hejailan,Partner,Freshfields Bruckhaus Deringer LLP in associat...,Riyadh,null,+966 11 479 2200,+966 1 479 1717,fares.al-hejailan@freshfields.com
1,https://www.freshfields.com/en-gb/contacts/fin...,David Almroth,Partner,"601 Lexington Avenue, 31st Floor New York, NY ...",New York,null,+1 212 277 4070,+1 646 521 5670,david.almroth@freshfields.com
2,https://www.freshfields.com/en-gb/contacts/fin...,Aled Batey,Partner,100 Bishopsgate London EC2P 2SR,London,null,+44 20 7785 2481,+44 20 7108 2481,aled.batey@freshfields.com
3,https://www.freshfields.com/en-gb/contacts/fin...,Simone Bono,Partner,100 Bishopsgate London EC2P 2SR,London,null,+44 20 7832 7269,+44 20 7832 7001,simone.bono@freshfields.com
4,https://www.freshfields.com/en-gb/contacts/fin...,Christopher Davis,Partner,100 Bishopsgate London EC2P 2SR,London,null,+44 20 7936 4000,+44 20 7832 7001,christopher.davis@freshfields.com
5,https://www.freshfields.com/en-gb/contacts/fin...,Thijs P. Flokstra,Partner/Notaris,Strawinskylaan 10 1077 XZ Amsterdam,Amsterdam,+31 6 53 17 33 28,+31 20 4857620,+31 20 485 7001,thijs.flokstra@freshfields.com
6,https://www.freshfields.com/en-gb/contacts/fin...,Daniel French,Partner,"55th Floor, One Island East Taikoo Place, Quar...",Hong Kong,null,+852 2913 2600,+852 2810 6192,daniel.french@freshfields.com
7,https://www.freshfields.com/en-gb/contacts/fin...,Iñaki Gabilondo,Partner,"Torre Europa Paseo de la Castellana, 95 Madrid...",Madrid,null,+34 91 700 3700,+34 91 308 4636,inaki.gabilondo@freshfields.com
8,https://www.freshfields.com/en-gb/contacts/fin...,Andrew Hagan,Partner,100 Bishopsgate London EC2P 2SR,London,null,+44 20 7832 7434,+44 20 7108 7434,andrew.hagan@freshfields.com
9,https://www.freshfields.com/en-gb/contacts/fin...,Dr. Mario Hüther,Partner,Bockenheimer Anlage 44 60322 Frankfurt am Main,Frankfurt am Main,null,+49 69 27 30 80,+49 69 23 26 64,mario.huether@freshfields.com


In [7]:
app = FlaskAPI(__name__)
 
api = Api(app)
e = create_engine('sqlite:///selenium_flask_sqlite.db')


@app.route('/create_table', methods=['POST']) #need to pass the table name Eg: /create_table?tbl=tbl_name
def create_table():
    try:
        tbl = request.args.get('tbl')
        conn = e.connect()
        conn.execute('CREATE TABLE IF NOT EXISTS {} (SNO INTEGER PRIMARY KEY AUTOINCREMENT,URL TEXT,NAME TEXT,ROLE TEXT,ADDRESS TEXT,COUNTRY TEXT,MOBILE TEXT,TELEPHONE TEXT,FAX TEXT,EMAIL TEXT);'.format(tbl))
        text = {"Message " : "Table created successfully"}
        return text
    except Exception as error:
        text = {"Message " : "Table creation failed : {}".format(error)}
        return text
    finally:
        conn.close()

@app.route('/tbl_view_all', methods=['GET']) #Eg: /tbl_view_all?tbl=tbl_name
def tbl_view_all():
    try:
        tbl = request.args.get('tbl')
        conn=e.connect()
        result = conn.execute("select * from {}".format(tbl))
        return json.dumps([dict(r) for r in result])
    except Exception as error:
        text = {"Message " : "Records not found : {}".format(error)}
        return text
    finally:
        conn.close()
        
@app.route('/tbl_view_one', methods=['GET']) #Eg: /tbl_view_one?tbl=tbl_name&url=https://www.freshfields.com/en-gb/contacts/find-a-lawyer/f/french-daniel/
def tbl_view_one():
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        url = all_args.get('url')
        result = conn.execute("select * from {} where URL = '{}';".format(tbl,url))  
        return json.dumps([dict(r) for r in result])
    
    except Exception as error:
        text = {"Message " : "Record not found : {}".format(error)}
        return text    

@app.route('/freshfields_insert', methods=['POST'])
def freshfields_insert():    
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        data.to_sql(tbl, conn, if_exists='append',index=False)
        text = {"Message " : "Records inserted successfully"}
        return text
    
    except Exception as error:
        text = {"Message " : "Insertion failed : {}".format(error)}
        return text

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2021 15:29:25] "GET /create_table?tbl=selenium HTTP/1.1" 405 -
127.0.0.1 - - [21/Nov/2021 15:29:30] "POST /create_table?tbl=selenium HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 15:31:21] "POST /create_table?tbl=test HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 15:31:32] "GET /tbl_view_all?tbl=test HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 15:31:40] "GET /tbl_view_one?tbl=test&url=https://www.freshfields.com/en-gb/contacts/find-a-lawyer/a/almroth-david/ HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 15:31:57] "POST /bulk_insert?tbl=bulk_tbl HTTP/1.1" 404 -
127.0.0.1 - - [21/Nov/2021 15:32:07] "POST /bulk_insert?tbl=test HTTP/1.1" 404 -
127.0.0.1 - - [21/Nov/2021 15:32:13] "POST /freshfields_insert?tbl=test HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 15:32:17] "GET /tbl_view_all?tbl=test HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 15:32:26] "GET /tbl_view_one?tbl=test&url=https://www.freshfields.com/en-gb/contacts/